# Road Traffic Accidents in Switzerland

Our project goal is to scrap all traffic accidents from the accidents map from http://map.donneesaccidents.ch/

## Data scraping strategy

Accessing http://map.donneesaccidents.ch/, wich redirects to : <br>
https://map.geo.admin.ch/?topic=vu&lang=fr&bgLayer=ch.swisstopo.pixelkarte-grau&layers=ch.astra.unfaelle-personenschaeden_alle&layers_timestamp=&catalogNodes=1318


Postman parses the following parameters : 
<code>
topic:vu
lang:en
bgLayer:ch.swisstopo.pixelkarte-grau
layers:ch.astra.unfaelle-personenschaeden_alle
layers_timestamp:
catalogNodes:1318
</code>

The most important one is layers:ch.astra.unfaelle-personenschaeden_alle.<br>
It is the layer that contains all the geo-information dots on "Accidents with personal injury" which is the selected data layer.
<img src="Resources/images/layer_selector.png">

Selection all kinds of accidents returns the following :<br>
<img src="Resources/images/layer_selector_all.png">
with layer parameters :<br>
layers:<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_alle,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_getoetete,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fussgaenger,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_fahrraeder,<br>
    &nbsp;ch.astra.unfaelle-personenschaeden_motorraeder<br>
layers_timestamp:,,,,<br>

Now we want every data for each layer. By selecting a dot on the map, it queries the related data to the server.
What we want to do is selecting all the entries in the map to retrieves all data. This is done by ctrl clicking the whole area.

This makes a query for each "layers" parameter :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:<i>LAYER_PARAM</i>
mapExtent:269999.04209536605,9501.673331833561,1037999.042095366,389001.67333183356
returnGeometry:true
tolerance:5
</code><br>
But doesn't select all dots on map, so let's try the "load more results" button on a 'accidetns with fatalities' layer, we get :
<code>
geometry:443999.04209536605,39001.6733318335,870499.0420953662,303001.67333183356
geometryFormat:geojson
geometryType:esriGeometryEnvelope
imageDisplay:1536,759,96
lang:en
layers:all:ch.astra.unfaelle-personenschaeden_getoetete
mapExtent:136199.04209536605,-28148.32666816644,1134599.042095366,465201.67333183356
<b>offset:200</b>
returnGeometry:true
tolerance:5
</code>
Pressing load more until no more possible give offset=1200 (for a total of 1337 objects) i.e it loads data entries 200 by 200

## JSON Data scraping

In [1]:
import requests
import json

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from Scripts.helpers import *
from Scripts.plots import *


import pprint
#from bs4 import BeautifulSoup

In [2]:
pp = pprint.PrettyPrinter(indent=4)

In [3]:
#import raw data
data = import_data(all_data = False)

Processing layer : ch.astra.unfaelle-personenschaeden_alle
Layer processed : 804 records

Processing layer : ch.astra.unfaelle-personenschaeden_getoetete
Layer processed : 804 records

Processing layer : ch.astra.unfaelle-personenschaeden_fussgaenger
Layer processed : 804 records

Processing layer : ch.astra.unfaelle-personenschaeden_fahrraeder
Layer processed : 804 records

Processing layer : ch.astra.unfaelle-personenschaeden_motorraeder
Layer processed : 804 records

Sample dataset processed : 4020 records



In [4]:
json_data_preprocessed = preprocess_data(data)

In [5]:
print("Data entry example after clean and reformat:\n")
json_data_preprocessed[0]

Data entry example after clean and reformat:



{'accidenttype_fr': 'accident par tamponnement',
 'accidenttypecode': 2,
 'accidentyear': 2011,
 'canton': 'SO',
 'coordinates': [606606.0, 228160.0],
 'day': 'lundi',
 'fsocommunecode': '2601',
 'id': 'A62ACA426CC7E02AE0430A865E33E02A',
 'label': 'Auffahrunfall',
 'layerName': 'Accidents avec dommages corporels',
 'month': 'juin',
 'roadtype_fr': 'route principale',
 'roadtypecode': 432,
 'severitycategory_fr': 'accident avec blessés légers',
 'severitycategorycode': 'ULV',
 'time': '18h-19h'}

In [6]:
df = pd.DataFrame.from_dict(json_data_preprocessed)
df.set_index('id', inplace=True)
df.columns = ['accident_type','accident_type_code','accident_year','canton','coordinates','day','fsocommunecode','label','layerName','month','road_type','road_type_code','severity_category','severity_category_code','time']
df.sample(5)

,accident_type,accident_type_code,accident_year,canton,coordinates,day,fsocommunecode,label,layerName,month,road_type,road_type_code,severity_category,severity_category_code,time
id,,,,,,,,,,,,,,,
F0F7A9E5754A0156E0430A83942784FB,accident impliquant des piétons,8,2013,VS,"[595270.0, 120565.0]",samedi,6266,Fussgängerunfall,Accidents mortels,novembre,route principale,432,accident avec tués,UGT,19h-20h
233D65CC14FA0070E0530A83942794CB,accident en s'engageant sur une route,4,2015,ZH,"[687225.0, 263422.0]",mardi,0056,Einbiegeunfall,Accidents avec la part. de motos,septembre,route principale,432,accident avec blessés légers,ULV,18h-19h
242DD4AEB6710036E0530A839427492B,accident impliquant des piétons,8,2015,ZH,"[682669.0, 247745.0]",mercredi,0261,Fussgängerunfall,Accidents avec la part. de piétons,octobre,route principale,432,accident avec blessés graves,USV,22h-23h
2058B367FF64014AE0530A8394279839,accident impliquant des piétons,8,2015,BE,"[625764.0, 230393.0]",vendredi,0329,Fussgängerunfall,Accidents avec la part. de piétons,septembre,route principale,432,accident avec blessés graves,USV,15h-16h
2376EF9F4F3801DCE0530A83942775EC,accident lors d'un dépassement ou lors d'un ch...,1,2015,AG,"[652135.0, 250333.0]",mercredi,4206,Überholunfall oder Fahrstreifenwechsel,Accidents avec la part. de vélos,octobre,route principale,432,accident avec blessés graves,USV,10h-11h


In [7]:
plot_all_features(df)

Plotting all features
plotting feature accident_type
plotting feature accident_type_code
plotting feature accident_year
plotting feature canton
plotting feature coordinates
->    Type Error : unhashable type: 'list'
plotting feature day
plotting feature label
plotting feature layerName
->    Key Error : 'layerName'
plotting feature month
plotting feature road_type
plotting feature road_type_code
plotting feature severity_category
plotting feature severity_category_code
plotting feature time
Done plotting


# Data analysis

1) Accidents par rapport au temps<br>
2) Corrélation nombre/type d'accident avec les endroits (Valais ivresse)<br>
3) Tracker des anomalies (fin/début d'une série d'accident) et essayer d'en trouver la cause<br>

In [8]:
plot_feature_combination(df, ['month', 'accident_year'])

Plotting features : ['month', 'accident_year']
Done plotting month_accident_year


In [9]:
plot_feature_combination(df, ['day', 'canton'])

Plotting features : ['day', 'canton']
Done plotting day_canton


In [10]:
plot_feature_combination(df, ['day', 'road_type'])

Plotting features : ['day', 'road_type']
Done plotting day_road_type
